In [7]:
from HTMLParser import HTMLParser

class ExtractJSON(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.inScript = False
        self.count = 0
        self.vendorsJSON = ""
        self.productsJSON = ""

    def handle_starttag(self, tag, attrs):
        if tag == 'script':
            for name, value in attrs:
                if name == 'type' and value == 'text/javascript':
                    if name == 'language':
                        return
            self.inScript = True

    def handle_endtag(self, tag):
        if tag == "script" and self.inScript:
            self.inScript = False

    def handle_data(self, data):
        if self.inScript and data.startswith("var vendors"):
            self.vendorsJSON = data[data.index("{"):data.index(";")]
            self.productsJSON = data[data.index(";var proddata = ")+16:data.index("$( document )")]

raw_html = open('/media/camilo/KINGSTON/BlackMarkets/dreammarket/market_2014_07_15/index.html', 'r').read().replace('\n', '').replace('\t', '')
#raw_html = open('/media/camilo/KINGSTON/BlackMarkets/dreammarket/market_2014_07_15/index.html_subcatId=9', 'r').read().replace('\n', '').replace('\t', '')
#raw_html = open('/media/camilo/KINGSTON/BlackMarkets/dreammarket/market_2014_11_21/index.html_subcatId=9', 'r').read().replace('\n', '').replace('\t', '')


parser = ExtractJSON()
parser.feed(raw_html)
parser.vendorsJSON
parser.productsJSON

'[{"lid":"414","lnk":"648525.389603","title":"1.8oz Grade A Indoor \\"Bud-Shake\\"","imageId":"821251.874965","sellerOutLink":-1888529356,"btcPriceText":"0.325","localPriceText":"$200","shipsFrom":"\\"Canada\\"","isTrustedSeller":false},{"lid":"387","lnk":"375979.200345","title":"CRITICAL MASS weed 10g (7usd\\/g)","imageId":"532683.693240","sellerOutLink":1961780166,"btcPriceText":"0.1138","localPriceText":"$70","shipsFrom":"\\"Spain\\"","isTrustedSeller":false},{"lid":"400","lnk":"652976.926096","title":"1oz Grade A Hash","imageId":"502134.758175","sellerOutLink":-1888529356,"btcPriceText":"0.3659","localPriceText":"$225","shipsFrom":"\\"Canada\\"","isTrustedSeller":false},{"lid":"230","lnk":"29225.981209","title":"Speed Paste 1g 72 % ","imageId":"885284.67198","sellerOutLink":1210337208,"btcPriceText":"0.0223","localPriceText":"&euro;10.23","shipsFrom":"\\"Germany\\"","isTrustedSeller":false},{"lid":"386","lnk":"583856.403598","title":"BLACK JACK weed 10g (7usd\\/g)","imageId":"99227

In [8]:
class Vendors(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.recording = False
        self.vendor = ""

    def handle_starttag(self, tag, attrs):
        if tag == 'a':
            for name, value in attrs:
                if name == 'href' and value.startswith("./contactMember?member"):
                    self.recording = True

    def handle_endtag(self, tag):
        if tag == "a" and self.recording:
            self.recording = False

    def handle_data(self, data):
        if self.recording:
            self.vendor = data

In [9]:
class Category(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.recording = False
        self.result = ""

    def handle_starttag(self, tag, attrs):
        if tag == 'title':
            self.recording = True

    def handle_endtag(self, tag):
        if tag == "title" and self.recording:
            self.recording = False

    def handle_data(self, data):
        if self.recording:
            self.result = data
            
raw_title_html = open('/media/camilo/KINGSTON/BlackMarkets/dreammarket/market_2014_07_15/index.html', 'r').read().replace('\n', '').replace('\t', '')
#raw_title_html = open('/media/camilo/KINGSTON/BlackMarkets/dreammarket/market_2014_07_15/index.html_subcatId=9', 'r').read().replace('\n', '').replace('\t', '')
#raw_title_html = open('/media/camilo/KINGSTON/BlackMarkets/dreammarket/market_2014_11_21/index.html_subcatId=9', 'r').read().replace('\n', '').replace('\t', '')

parser_category = Category()
parser_category.feed(raw_title_html)
parser_category.result

'Dream Market'

In [78]:
import json

products = json.loads(parser.productsJSON)
vendors = json.loads(parser.vendorsJSON)

for current_prod in products:
    print current_prod["lid"]
    print current_prod["lnk"]
    print current_prod["title"]
    print current_prod["btcPriceText"]
    print current_prod["localPriceText"]
    print parser_title.category

    vendors_parser = Vendors()
    vendors_parser.feed(vendors[str(current_prod["sellerOutLink"])])
    
    print vendors_parser.vendor
    print current_prod["shipsFrom"]

414
648525.389603
1.8oz Grade A Indoor "Bud-Shake"
0.325
$200
Dream Market
 bcdirect   
"Canada"
387
375979.200345
CRITICAL MASS weed 10g (7usd/g)
0.1138
$70
Dream Market
 mendosa   
"Spain"
400
652976.926096
1oz Grade A Hash
0.3659
$225
Dream Market
 bcdirect   
"Canada"
230
29225.981209
Speed Paste 1g 72 % 
0.0223
&euro;10.23
Dream Market
 sthompson   
"Germany"
386
583856.403598
BLACK JACK weed 10g (7usd/g)
0.1138
$70
Dream Market
 mendosa   
"Spain"
234
460341.746438
1G HIGH CLASS TAN MDMA
0.0599
&euro;27.5
Dream Market
 sthompson   
"germany"
402
165615.943286
1.8oz AAA Bubba Kush Indoor BC Bud
0.65
$400
Dream Market
 bcdirect   
"Canada"
229
68487.976575
Speed Paste 2g 72 % 
0.0414
&euro;19
Dream Market
 sthompson   
"Germany"
390
134949.560244
CREAM CARAMEL weed 10g (7usd/g)
0.1138
$70
Dream Market
 mendosa   
"Spain"
403
725246.481618
1oz AAA Critical Plus BC Bud Greenhouse
0.325
$200
Dream Market
 bcdirect   
"Canada"
405
674472.32404
1oz Vintage 2006 AAA Greenhouse BC Bud
0.3

In [15]:
import sys  

reload(sys)  
sys.setdefaultencoding('utf8')

import os
import cPickle
from tempfile import TemporaryFile
from xlwt import Workbook
import glob
import json

book = Workbook(encoding='utf-8')
sheet1 = book.add_sheet('Sheet 1')

files_list = glob.glob('/media/camilo/KINGSTON/BlackMarkets/dreammarket/market_*/index.html*')

count = 0

for current_file in files_list:
    complete_file = current_file
    raw_html = open(complete_file, 'r').read().replace('\n', '').replace('\t', '')
    
    parserCategory = Category()
    parserCategory.feed(raw_html)

    parser_JSON = ExtractJSON()
    parser_JSON.feed(raw_html)

    try:
        products = json.loads(parser_JSON.productsJSON)
        vendors = json.loads(parser_JSON.vendorsJSON)
    
        for current_prod in products:
            vendors_parser = Vendors()
            vendors_parser.feed(vendors[str(current_prod["sellerOutLink"])])
            sheet1.write(count,0,complete_file)
            sheet1.write(count,1,current_prod["lid"])
            sheet1.write(count,2,current_prod["lnk"])
            sheet1.write(count,3,current_prod["title"])
            sheet1.write(count,4,current_prod["btcPriceText"])
            sheet1.write(count,5,current_prod["localPriceText"])
            sheet1.write(count,6,parser_category.result)
            sheet1.write(count,7,vendors_parser.vendor)
            sheet1.write(count,8,current_prod["shipsFrom"])
            count+=1
    except ValueError, e:
        sheet1.write(count,0,complete_file)
        count+=1

book.save('Price_Analysis_dreammarket_1_all.xls')
book.save(TemporaryFile())